# Multi_Qubit_Systems

<a id="0"></a> <br>
1. [Codercise I.11.1 - Preparing basis state](#1)
2. [Codercise I.11.2  - Separable Operations](#2)
3. [Codercise I.11.3 - Expectation value of two-qubit observable](#3)
4. [Codercise I.11.4 - Double Trouble](#4)

In [2]:
import numpy as np
import pennylane as qml

<a id="1"></a>
# Codercise I.11.1 - Preparing basis state

In [3]:
num_wires = 3
dev = qml.device("default.qubit", wires=num_wires)


@qml.qnode(dev)
def make_basis_state(basis_id):
    """Produce the 3-qubit basis state corresponding to |basis_id>.

    Note that the system starts in |000>.

    Args:
        basis_id (int): An integer value identifying the basis state to construct.

    Returns:
        np.array[complex]: The computational basis state |basis_id>.
    """

    bits = [int(x) for x in np.binary_repr(basis_id, width=num_wires)]
    qml.BasisStatePreparation(bits, wires=[x for x in range(num_wires)])
    return qml.state()


basis_id = 3
print(f"Output state = {make_basis_state(basis_id)}")


Output state = [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]



<a id="2"></a>
# Codercise I.11.2  - Separable Operations

In [4]:
# Creates a device with *two* qubits
dev = qml.device("default.qubit", wires=2)


@qml.qnode(dev)
def two_qubit_circuit():
    # PREPARE |+>|1>
    qml.Hadamard(0)
    qml.PauliX(1)
    # RETURN TWO EXPECTATION VALUES, Y ON FIRST QUBIT, Z ON SECOND QUBIT

    return (qml.expval(qml.PauliY(0)), qml.expval(qml.PauliZ(1)))


print(two_qubit_circuit())


(tensor(0., requires_grad=True), tensor(-1., requires_grad=True))



<a id="3"></a>
# Codercise I.11.3 - Expectation value of two-qubit observable

In [5]:
dev = qml.device("default.qubit", wires=2)


@qml.qnode(dev)
def create_one_minus():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE |1>|->
    qml.PauliX(0)
    qml.PauliX(1)
    qml.Hadamard(1)
    # RETURN A SINGLE EXPECTATION VALUE Z \otimes X
    expect_value = qml.expval(qml.PauliZ(0)@qml.PauliX(1))

    return expect_value


print(create_one_minus())


0.9999999999999996



<a id="4"></a>
# Codercise I.11.4 - Double Trouble

In [ ]:
dev = qml.device("default.qubit", wires=2)


@qml.qnode(dev)
def circuit_1(theta):
    """Implement the circuit and measure Z I and I Z.

    Args:
        theta (float): a rotation angle.

    Returns:
        float, float: The expectation values of the observables Z I, and I Z
    """
    qml.RX(theta, wires=0)
    qml.RY(2*theta, wires=1)

    return (qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(1)))


@qml.qnode(dev)
def circuit_2(theta):
    """Implement the circuit and measure Z Z.

    Args:
        theta (float): a rotation angle.

    Returns:
        float: The expectation value of the observable Z Z
    """
    qml.RX(theta, wires=0)
    qml.RY(2*theta, wires=1)

    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


def zi_iz_combination(ZI_results, IZ_results):
    """Implement a function that acts on the ZI and IZ results to
    produce the ZZ results. How do you think they should combine?

    Args:
        ZI_results (np.array[float]): Results from the expectation value of
            ZI in circuit_1.
        IZ_results (np.array[float]): Results from the expectation value of
            IZ in circuit_2.

    Returns:
        np.array[float]: A combination of ZI_results and IZ_results that
        produces results equivalent to measuring ZZ.
    """

    combined_results = np.zeros(len(ZI_results))
    combined_results = np.multiply(ZI_results, IZ_results)

    return combined_results


theta = np.linspace(0, 2 * np.pi, 100)

# Run circuit 1, and process the results
circuit_1_results = np.array([circuit_1(t) for t in theta])

ZI_results = circuit_1_results[:, 0]
IZ_results = circuit_1_results[:, 1]
combined_results = zi_iz_combination(ZI_results, IZ_results)

# Run circuit 2
ZZ_results = np.array([circuit_2(t) for t in theta])

# Plot your results
plot = plotter(theta, ZI_results, IZ_results, ZZ_results, combined_results)
